# Introduction to Bayesian Statistics

In these notes we introduce the basic concepts of Bayesian statistics, and highlight the key differences with the classical approach. 

### Linear regression from the classical perspective

In these notes we consider the *simple linear regression* which refers to a linear regression model with only a single predictor, or input variable $x$, a single output variable $y$, and Gaussian distributed model errors. The model is written as follows

$$ y = \beta_0 + \beta_1x + \epsilon, $$

where $\beta_0$ and $\beta_1$ are the parameters and $\epsilon$ is a Gaussian variable with zero mean and variance $\sigma^2$, also written as $\epsilon \sim N(0,\sigma^2)$. In the classical approach we estimate the parameters of the model via maximum likelihood, which has the familiar closed form solution for the best fitting parameters

$$ \hat{\mathbf{\beta}} = (X'X)^{-1}X'y  $$

with 

$$ \hat{\sigma^2} = \frac{1}{N-1} \sum_{i=1}^{N} (y_i - \hat{\beta_0} + \hat{\beta_1}x_i)^2  $$

and 'standard error'

$$  se(\mathbf{\beta}) = \frac{\hat{\sigma}}{\sum_{i=1}^N (x_i - \bar{x})^2}.$$

PLACEHOLDER: hypothesis test

### Linear regression from the Bayesian perspective

We can write the same model in hierarchical Bayesian form

$$ p(y | x, \beta) \sim N(\beta_0 + \beta_1x,\sigma^2) $$

$$ p(\beta_0) \sim uniform(a_1,b_1) $$

$$ p(\beta_1) \sim uniform(a_2,b_2) $$

and fit the model in Stan.

## The influence and uses of priors

To demonstrate the utility of priors, we now consider a hypothetical scientific scenario. We consider the case where we have some previous knowledge about the key model parameter $\beta_1$. For example, a previous study may have used a dataset from another area in the world and you'd like to update the value of the parameter using the new data from California. Or perhaps there is chemical theory that predicts what the value should be. In the case of the former, let's assume a previous study has estimate the value as $\beta_1 = 5x10^{-3},$ with $se(\beta_1) = 1x10^{-3}$. We specify this as a prior in the Stan code and define a new model *SLR_prior*. 



In [ ]:
SLR <- "data{
	int N; 			//length of dataset
	vector[N] y;	//response observations
	vector[N] x;	//input observations
}
parameters{
	real beta0;
	real beta1;
	real<lower=0> sigma;
}
model{
	beta0 ~ uniform(-1,1);				//prior on beta0, uniform density on (-1,1)
	beta1 ~ uniform(-1,1);				//prior on beta1
	y ~ normal(beta0 + beta1*x, sigma);	//likelihood model; p(y|x,beta0,beta1,sigma) ~ Gaussian 
}
generated quantities{
	vector[N] y_pred;     //predictive distribution
	vector[N] log_lik;    //marginal log likelihood for model comparison later on

	for (i in 1:N)
		y_pred[i] = beta0 + beta1*x[i]; //generate prediction for y within MCMC step for parameters
	for (i in 1:N)
		log_lik[i] = normal_lpdf(y[i] | beta0 + beta1*x[i], sigma);  //compute marginal log-likelihood within MCMC
}"

In [ ]:
SLR_prior <- "data{
	int N; 			//length of dataset
	vector[N] y;	//response observations
	vector[N] x;	//input observations
}
parameters{
	real beta0;
	real beta1;
	real<lower=0> sigma;
}
model{
	beta0 ~ uniform(-1,1);                  //prior on beta0, uniform density on (-1,1)
	beta1 ~ normal(1E-4,0.5E-4);              //prior on beta1
	y ~ normal(beta0 + beta1*x, sigma);	    //likelihood model; p(y|x,beta0,beta1,sigma) ~ Gaussian 
}
generated quantities{
	vector[N] y_pred;     //predictive distribution
	vector[N] log_lik;    //marginal log likelihood for model comparison later on

	for (i in 1:N)
		y_pred[i] = beta0 + beta1*x[i];                              //generate prediction for y within MCMC step for parameters
	for (i in 1:N)
		log_lik[i] = normal_lpdf(y[i] | beta0 + beta1*x[i], sigma);  //compute marginal log-likelihood within MCMC
}"

In [ ]:
library(rstan)                                #load package
rstan_options(auto_write=TRUE)                # 
options(mc.cores = parallel::detectCores())   #allows stan to utilize multiple cores

In [ ]:
mod_slr        <- stan_model(model_code=SLR)        #compile the C++ program
mod_slr_prior  <- stan_model(model_code=SLR_prior)        #compile the C++ program

In [ ]:
datmp  <- read.csv('D:/Dropbox/Teaching/Bayes/DSI/data/ozone_Riverside_epa.csv')    #read data as .csv file
datmp  <- datmp[complete.cases(cbind(datmp$temp,datmp$Arithmetic.Mean)),]           #subset for ozone and temperature columns
stan_dat <- list(y=datmp$Arithmetic.Mean[1:200], x=datmp$temp[1:200], N=200)        #subsample larger dataset into a dataframe for stan

In [ ]:
fit_slr_prior  <- sampling(mod_slr_prior, data=stan_dat)  #MCMC sampling
post_slr_prior <- extract(fit_slr_prior)  

In [ ]:
par(mfrow=c(2,2))
plot(seq(-0.05,0.05,length.out=10), rep(0.5,10),type='l',ylim=c(0,4),ylab='',xlab='beta0',bty='n',main='beta0') 
    par(new=TRUE)
    hist(post_slr_prior$beta0,xlim=c(-0.05,0.05),main='',xlab='',ylab='',xaxt='n',yaxt='n',bty='n',col=adjustcolor('black',alpha.f=0.5),bty='n',xaxt='n',yaxt='n')
    abline(v=0,lty=2)

##-Plot the naive estimate-##
    par(new=TRUE)
    hist(post$beta0,xlim=c(-0.05,0.05),main='',xlab='',ylab='',xaxt='n',yaxt='n',bty='n')

plot(seq(-0.00075,0.00075,length.out=10), rep(0.5,10),type='l',ylim=c(0,4),bty='n',ylab='',xlab='beta1',main='beta1')
	par(new=TRUE)
    hist(post_slr_prior$beta1,xlim=c(-0.00075,0.00075),main='',xlab='',ylab='',xaxt='n',yaxt='n',bty='n',col=adjustcolor('black',alpha.f=0.5))
	abline(v=0,lty=2)

##-Plot the prior-##
    par(new=TRUE)
    x <- seq(-0.00075,0.00075,length.out=1000)
    plot(x,dnorm(x,mean=1E-4,sd=0.5E-4),type='l',lwd=2,bty='n',xaxt='n',yaxt='n',ylab='',xlab='')
##-Plot the naive estimate-##
	par(new=TRUE)
    hist(post$beta1,xlim=c(-0.00075,0.00075),main='',xlab='',ylab='',xaxt='n',yaxt='n',bty='n',xaxt='n',yaxt='n')

plot(seq(0,0.015,length.out=10), rep(0.5,10),type='l',ylim=c(0,4),bty='n',ylab='',xlab='sigma',main='sigma')
	par(new=TRUE)
    hist(post_slr_prior$sigma,xlim=c(0,0.015),main='',xlab='',ylab='',xaxt='n',yaxt='n',bty='n',col=adjustcolor('black',alpha.f=0.5))
	par(new=TRUE)
    hist(post$sigma,xlim=c(0,0.015),main='',xlab='',ylab='',xaxt='n',yaxt='n',bty='n')

plot(stan_dat$x, stan_dat$y,xlab='Temperature [F]',ylab='Ozone [umol/kg]')
    lines(seq(30,85),quantile(post$beta0,0.05) + quantile(post$beta1,0.05)*seq(30,85),lty=2)
    lines(seq(30,85),quantile(post$beta0,0.50) + quantile(post$beta1,0.50)*seq(30,85),lwd=2)
    lines(seq(30,85),quantile(post$beta0,0.95) + quantile(post$beta1,0.95)*seq(30,85),lty=2)